In [1]:
%pip -q install google-genai

In [2]:
# Configurando a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configurando o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalando Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
# Instalando Framework ADK de agentes do Google
!pip install -q google-adk

# Importações necessárias
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types as genai_types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings

warnings.filterwarnings("ignore")

print("Google ADK instalado e bibliotecas importadas.")

Google ADK instalado e bibliotecas importadas.


In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = genai_types.Content(role="user", parts=[genai_types.Part(text=message_text)]) # Adaptado aqui

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              # final_response += "\n" # Removido para evitar duplas quebras de linha, a saída do LLM geralmente já tem
    return final_response

print("Função call_agent definida.")

Função call_agent definida.


In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("Função to_markdown definida.")

Função to_markdown definida.


In [26]:
###################################################
# --- Agente 1: Pesquisador de Temas em Alta --- #
###################################################
def agente_pesquisador_temas(nicho_usuario: str, data_de_hoje: str) -> str:
    """
    Este agente usa a busca do Google para encontrar temas recentes e relevantes
    dentro de um nicho especificado, retornando de 3 a 5 temas promissores
    para vídeos curtos.

    Args:
        nicho_usuario: O nicho de interesse fornecido pelo usuário.
        data_de_hoje: A data atual, para dar contexto de tempo ao agente.

    Returns:
        Uma string contendo a lista de temas sugeridos e suas justificativas.
    """
    # Define o agente com nome, modelo, descrição, ferramentas e instruções (prompt)
    pesquisador_temas = Agent(
        name="agente_pesquisador_temas_v1", # Nome descritivo para o agente
        model=MODEL_ID, # Modelo Gemini a ser usado
        description="Agente especialista em encontrar temas recentes e relevantes para vídeos curtos com base em um nicho.",
        tools=[google_search], # Ferramenta de busca do Google disponibilizada para o agente
        instruction=f"""
        Você é um especialista em tendências de conteúdo para vídeos curtos (TikTok, Reels).
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar os tópicos MAIS RECENTES (últimas 2-4 semanas a partir de {data_de_hoje}) e RELEVANTES dentro do nicho: "{nicho_usuario}".
        Filtre e identifique de 3 a 5 temas textuais que parecem promissores para criar vídeos curtos virais ou com alto engajamento.
        Apresente os temas em uma lista numerada, cada um com uma breve justificativa (1-2 frases) do porquê é promissor.
        Exemplo de saída:
        1. [Tema 1]: [Justificativa breve]
        2. [Tema 2]: [Justificativa breve]
        ...
        """
    )

    # Define a mensagem de entrada para o agente.
    # Embora as informações principais já estejam no prompt, uma mensagem explícita pode ajudar.
    entrada_do_agente = f"Nicho: {nicho_usuario}\nData de hoje: {data_de_hoje}\nPor favor, encontre os temas em alta para vídeos curtos."

    print(f"\n🔎 Agente Pesquisador de Temas: Buscando temas para o nicho '{nicho_usuario}'...")
    # Chama o agente usando a função auxiliar e retorna a resposta
    temas_encontrados = call_agent(pesquisador_temas, entrada_do_agente)
    return temas_encontrados

# print("Função agente_pesquisador_temas definida.") # Se quiser confirmação

In [27]:
###################################################
# --- Agente 2: Gerador de Ângulos Criativos (v2)--- #
###################################################
def agente_gerador_angulos(tema_escolhido: str) -> str:
    """
    Este agente recebe um tema e sugere 3 ângulos ou formatos de vídeo
    criativos para abordar esse tema, seguindo um formato de saída específico.

    Args:
        tema_escolhido: O tema selecionado pelo usuário.

    Returns:
        Uma string contendo 3 sugestões de ângulos, cada um com título e descrição.
    """
    gerador_angulos = Agent(
        name="agente_gerador_angulos_v2",
        model=MODEL_ID,
        description="Agente criativo que sugere diferentes ângulos e formatos para vídeos curtos sobre um tema específico.",
        tools=[], # Não necessita de ferramentas externas, foca na criatividade do LLM.
        instruction=f"""
        Você é um roteirista criativo especializado em vídeos curtos para plataformas como TikTok e Reels.
        Seu objetivo é gerar ideias de ângulos e formatos de vídeo.

        Dado o tema: "{tema_escolhido}"

        Sugira 3 ângulos ou formatos de vídeo distintos e criativos para abordar este tema.
        Para cada ângulo, siga **EXATAMENTE** o formato abaixo:

        **ANGULO X:** [Aqui vai o TÍTULO CURTO E CHAMATIVO do ângulo/formato]
        **DESCRIÇÃO:** [Aqui vai a DESCRIÇÃO BREVE (1-2 frases) explicando a ideia central do vídeo para esse ângulo]

        --- (use esta linha com três traços para separar cada ângulo) ---

        Pense em formatos populares e engajadores.
        Apresente os 3 ângulos. Não numere os ângulos com 1, 2, 3. Use "ANGULO X" como identificador.

        Exemplo de Saída Esperada (apenas para ilustrar a estrutura de UM ângulo):
        **ANGULO 1:** Este é o Título do Primeiro Ângulo
        **DESCRIÇÃO:** Esta é a breve descrição do que seria o vídeo para o primeiro ângulo.
        --- (esta linha separadora é importante entre cada bloco de ângulo)

        **ANGULO 2:** Outro Título Criativo Para o Segundo Ângulo
        **DESCRIÇÃO:** E aqui a descrição para este segundo ângulo, explicando a ideia.
        --- (esta linha separadora é importante entre cada bloco de ângulo)

        **ANGULO 3:** Título Final Para o Terceiro Ângulo
        **DESCRIÇÃO:** Descrição final para a terceira sugestão de vídeo.
        --- (esta linha separadora é importante entre cada bloco de ângulo)
        """
    )

    entrada_do_agente = f"Tema fornecido: {tema_escolhido}\nPor favor, gere 3 ângulos criativos para este tema seguindo o formato especificado."

    print(f"\n🧠 Agente Gerador de Ângulos (v2): Criando ângulos para o tema '{tema_escolhido}'...")
    angulos_sugeridos = call_agent(gerador_angulos, entrada_do_agente)
    return angulos_sugeridos

# print("Função agente_gerador_angulos (AJUSTADA v2) definida.")

In [28]:
##############################################
# --- Agente 3: Roteirista Expresso (v2) --- #
##############################################
def agente_roteirista_expresso(tema_escolhido: str, angulo_escolhido: str) -> str:
    """
    Este agente cria um roteiro conciso para um vídeo curto, com Gancho,
    Desenvolvimento textual e CTA, além de uma sugestão de legenda,
    baseado no tema e ângulo escolhidos.

    Args:
        tema_escolhido: O tema do vídeo.
        angulo_escolhido: O ângulo/formato criativo para o vídeo.

    Returns:
        Uma string contendo o roteiro bruto e a sugestão de legenda.
    """
    roteirista = Agent(
        name="agente_roteirista_expresso_v2",
        model=MODEL_ID,
        description="Agente que cria roteiros concisos e dinâmicos para vídeos curtos (TikTok/Reels) com base em um tema e ângulo.",
        tools=[],
        instruction=f"""
        Você é um roteirista de alta performance, especialista em criar roteiros virais e engajadores para vídeos curtos (TikTok/Reels).
        Seu objetivo é fornecer um roteiro claro, conciso e com a estrutura ideal para prender a atenção, com texto pronto para ser narrado ou exibido.

        **Tema do Vídeo:** "{tema_escolhido}"
        **Ângulo/Formato Escolhido:** "{angulo_escolhido}"

        Crie um roteiro detalhado com os seguintes componentes:

        1.  **VOZ/TEXTO NA TELA - Gancho (Primeiros 3-5 segundos):**
            *   Uma frase ou pergunta extremamente impactante para prender a atenção IMEDIATAMENTE.

        2.  **VOZ/TEXTO NA TELA - Desenvolvimento (15-25 segundos):**
            *   **Escreva um texto fluido e contínuo que explique o conteúdo principal do vídeo.**
            *   Entregue valor ou entretenimento de forma clara, objetiva e rápida.
            *   **Evite apenas listar tópicos.** Conecte as ideias em frases completas, como uma narração.

        3.  **VOZ/TEXTO NA TELA - Chamada para Ação (CTA - Últimos 3-5 segundos):**
            *   Incentive o espectador a realizar uma ação específica.

        Além do roteiro, sugira um texto curto e chamativo para a **LEGENDA principal do vídeo** (sem hashtags).
        O tom deve ser adequado para o público jovem e dinâmico dessas plataformas.

        **Formato da Saída (use EXATAMENTE este formato, com os títulos em negrito):**
        ---
        **ROTEIRO DO VÍDEO:**

        **Gancho (3-5s):**
        [Aqui vai o texto para o gancho]

        **Desenvolvimento (15-25s):**
        [Aqui vai o TEXTO CONTÍNUO E FLUIDO do desenvolvimento, pronto para ser narrado.]

        **CTA (3-5s):**
        [Aqui vai o texto para a chamada para ação]

        ---
        **SUGESTÃO DE LEGENDA (sem hashtags):**
        [Aqui vai a sugestão de legenda cativante]
        ---
        """
    )

    entrada_do_agente = f"Tema: {tema_escolhido}\nÂngulo: {angulo_escolhido}\nPor favor, crie o roteiro com desenvolvimento textual e a sugestão de legenda."

    print(f"\n✍️ Agente Roteirista Expresso (v2): Criando roteiro para o tema '{tema_escolhido}' com o ângulo '{angulo_escolhido}'...")
    roteiro_bruto = call_agent(roteirista, entrada_do_agente)
    return roteiro_bruto

# print("Função agente_roteirista_expresso (AJUSTADA v2) definida.")

In [29]:
#######################################################
# --- Agente 4: Agente de Refinamento e Finalização (v2) --- #
#######################################################
def agente_refinamento_final(tema_escolhido: str, angulo_escolhido: str, roteiro_bruto_com_legenda: str) -> str:
    """
    Este agente refina um roteiro bruto, melhora a legenda, adiciona hashtags
    estratégicas (usando busca do Google) e opcionalmente sugere
    elementos visuais/sonoros.

    Args:
        tema_escolhido: O tema do vídeo.
        angulo_escolhido: O ângulo/formato do vídeo.
        roteiro_bruto_com_legenda: O roteiro e legenda gerados pelo Agente 3.

    Returns:
        Uma string contendo o roteiro finalizado, legenda otimizada, hashtags e sugestões bônus.
    """
    refinador = Agent(
        name="agente_refinamento_final_v2",
        model=MODEL_ID,
        description="Agente especialista em otimizar e finalizar roteiros de vídeos curtos, adicionando legendas cativantes e hashtags relevantes.",
        tools=[google_search], # Usa busca para encontrar hashtags relevantes
        instruction=f"""
        Você é um Editor Final e Especialista em Otimização de Conteúdo para TikTok e Reels.
        Você recebeu o seguinte rascunho de roteiro e sugestão de legenda para um vídeo sobre o tema "{tema_escolhido}" com o ângulo "{angulo_escolhido}":

        <<<RASCUNHO DO AGENTE ANTERIOR>>>
        {roteiro_bruto_com_legenda}
        <<<FIM DO RASCUNHO>>>

        Sua tarefa é refinar e finalizar este material:

        1.  **REVISAR E POLIR O ROTEIRO:**
            *   Verifique clareza, concisão, ritmo e fluxo.
            *   **No Desenvolvimento, assegure que o texto é fluido e narrativo, não apenas tópicos. Se necessário, reescreva para ser mais discursivo.**
            *   Ajuste o texto para ser dinâmico e com linguagem natural para vídeos curtos.
            *   Respeite os tempos sugeridos (Gancho ~3-5s, Desenvolvimento ~15-25s, CTA ~3-5s).

        2.  **REFINAR A LEGENDA:**
            *   Melhore a legenda para ser mais cativante e otimizada para engajamento.

        3.  **SUGERIR HASHTAGS ESTRATÉGICAS:**
            *   Forneça 4-6 hashtags relevantes (use `google_search` para pesquisar tendências para "{tema_escolhido}").
            *   Misture hashtags amplas e de nicho.

        4.  **SUGESTÕES VISUAIS/SONORAS (BÔNUS OPCIONAL):**
            *   Se pertinente, sugira 1-2 ideias de elementos visuais simples ou tipo de música de fundo.

        **Formato da Saída (use EXATAMENTE este formato):**
        ---
        **ROTEIRO FINALIZADO E OTIMIZADO:**

        **Gancho (3-5s):**
        [Texto do Gancho Refinado]

        **Desenvolvimento (15-25s):**
        [TEXTO CONTÍNUO E FLUIDO do Desenvolvimento Refinado]

        **CTA (3-5s):**
        [Texto da CTA Refinada]

        ---
        **LEGENDA FINAL E ENGAJADORA:**
        [Texto da Legenda Refinada e Otimizada]

        ---
        **HASHTAGS ESTRATÉGICAS:**
        #hashtag1 #hashtag2 #hashtag3 #hashtag4 #hashtag5

        ---
        **SUGESTÕES VISUAIS/SONORAS (BÔNUS):**
        - Visual: [Sugestão visual, se houver]
        - Som: [Sugestão de som/música, se houver]
        ---
        """
    )

    entrada_do_agente = f"Material para refinar:\nTema: {tema_escolhido}\nÂngulo: {angulo_escolhido}\nRoteiro e Legenda Brutos:\n{roteiro_bruto_com_legenda}\n\nPor favor, refine este material completamente, garantindo um desenvolvimento textual."

    print(f"\n✨ Agente de Refinamento (v2): Polindo o roteiro para '{tema_escolhido}'...")
    roteiro_finalizado = call_agent(refinador, entrada_do_agente)
    return roteiro_finalizado

# print("Função agente_refinamento_final (AJUSTADA v2) definida.")

In [32]:
# --- CÉLULA PRINCIPAL DE EXECUÇÃO DO SISTEMA DE AGENTES (AJUSTADA v2) ---

# Obtém a data atual formatada para usar no prompt do Agente 1
data_hoje_formatada = date.today().strftime("%d/%m/%Y")

print("🚀 Bem-vindo ao Criador de Roteiros para Vídeos Curtos! 🚀")
print("Este sistema utiliza uma cadeia de 4 agentes para gerar ideias e roteiros.\n")

# --- Coleta de Entrada do Usuário: Nicho ---
nicho = input("❓ Por favor, digite o NICHO sobre o qual você quer criar vídeos (ex: culinária vegana, games, reflexologia): ")

# Inicializa variáveis que armazenarão os resultados e escolhas
tema_escolhido_pelo_usuario = None
angulo_escolhido_pelo_usuario = None
roteiro_bruto_do_agente3 = None
roteiro_final_do_agente4 = None
prosseguir_para_proxima_etapa = False # Flag para controlar o fluxo

# Verifica se o nicho foi fornecido
if not nicho:
    print("⚠️ Você não digitou um nicho. O processo não pode continuar.")
else:
    print(f"\nÓtimo! Iniciando o processo para o nicho: '{nicho}'")

    # --- ETAPA 1: Agente Pesquisador de Temas ---
    print("\n--- Chamando Agente 1 (Pesquisador de Temas)... ---")
    temas_sugeridos_texto_completo = agente_pesquisador_temas(nicho, data_hoje_formatada)

    if temas_sugeridos_texto_completo:
        print("\n--- 💡 Resultado do Agente 1 (Pesquisador de Temas) ---")
        display(to_markdown(temas_sugeridos_texto_completo))

        temas_lista = []
        print("\n--- Processando temas para escolha... ---")
        try:
            for linha in temas_sugeridos_texto_completo.split('\n'):
                linha_strip = linha.strip()
                if linha_strip and linha_strip[0].isdigit() and '.' in linha_strip.split(' ', 1)[0]:
                    partes_tema = linha_strip.split('.', 1)[1].strip()
                    tema_texto = partes_tema.split(':', 1)[0].strip().replace('**', '')
                    if tema_texto: temas_lista.append(tema_texto)

            if temas_lista:
                print("\n✅ Temas extraídos com sucesso. Por favor, escolha um:")
                for i, t in enumerate(temas_lista): print(f"{i+1}. {t}")

                # Loop para obter a escolha do tema do usuário
                while True:
                    try:
                        escolha_num_tema_str = input(f"Digite o NÚMERO do tema escolhido (1-{len(temas_lista)}): ")
                        if not escolha_num_tema_str:
                            print("Nenhuma entrada detectada. Por favor, digite um número.")
                            continue
                        escolha_num_tema = int(escolha_num_tema_str)
                        if 1 <= escolha_num_tema <= len(temas_lista):
                            tema_escolhido_pelo_usuario = temas_lista[escolha_num_tema-1]
                            print(f"Você escolheu o tema: '{tema_escolhido_pelo_usuario}'")
                            prosseguir_para_proxima_etapa = True # Sinaliza para ir para o Agente 2
                            break # Sai do loop de escolha de tema
                        else:
                            print(f"Escolha inválida. Digite um número entre 1 e {len(temas_lista)}.")
                    except ValueError:
                        print("Entrada inválida. Por favor, digite um número.")
                    except Exception as e_input:
                        print(f"Erro inesperado durante o input do tema: {e_input}")
                        prosseguir_para_proxima_etapa = False
                        break # Sai do loop em caso de erro inesperado
            else:
                print("\n⚠️ Não consegui extrair uma lista clara de temas para escolha do Agente 1.")
                prosseguir_para_proxima_etapa = False
        except Exception as e_proc:
            print(f"\n⚠️ Ocorreu um erro ao tentar processar os temas: {e_proc}")
            prosseguir_para_proxima_etapa = False
    else:
        print("⚠️ Não foi possível encontrar temas com o Agente 1. Verifique sua conexão ou tente um nicho diferente.")
        prosseguir_para_proxima_etapa = False


    # --- ETAPA 2: Agente Gerador de Ângulos ---
    if prosseguir_para_proxima_etapa and tema_escolhido_pelo_usuario:
        prosseguir_para_proxima_etapa = False # Reseta o flag para a próxima escolha
        print("\n--- Chamando Agente 2 (Gerador de Ângulos)... ---")
        angulos_sugeridos_pelo_agente2 = agente_gerador_angulos(tema_escolhido_pelo_usuario)

        if angulos_sugeridos_pelo_agente2:
            print("\n--- 🧠 Resultado do Agente 2 (Gerador de Ângulos) ---")
            display(to_markdown(angulos_sugeridos_pelo_agente2))

            angulos_lista = []
            titulos_angulos_para_escolha = []
            print("\n--- Processando ângulos para escolha... ---")
            try:
                blocos_de_angulo = angulos_sugeridos_pelo_agente2.split('---')
                for bloco in blocos_de_angulo:
                    bloco_strip = bloco.strip()
                    if not bloco_strip: continue
                    titulo_encontrado = None
                    for linha_bloco in bloco_strip.split('\n'):
                        linha_limpa = linha_bloco.strip()
                        if linha_limpa.upper().startswith("ANGULO") or linha_limpa.upper().startswith("**ANGULO"):
                            titulo_encontrado = linha_limpa.split(':', 1)[1].strip().replace('**', '') if ':' in linha_limpa else linha_limpa
                            if titulo_encontrado:
                                angulos_lista.append(titulo_encontrado)
                                titulos_angulos_para_escolha.append(titulo_encontrado)
                            break
                if angulos_lista:
                    print("\n✅ Ângulos extraídos com sucesso. Por favor, escolha um:")
                    for i, a_titulo in enumerate(titulos_angulos_para_escolha): print(f"{i+1}. {a_titulo}")

                    while True:
                        try:
                            escolha_num_angulo_str = input(f"Digite o NÚMERO do ângulo escolhido (1-{len(angulos_lista)}): ")
                            if not escolha_num_angulo_str:
                                print("Nenhuma entrada detectada. Por favor, digite um número.")
                                continue
                            escolha_num_angulo = int(escolha_num_angulo_str)
                            if 1 <= escolha_num_angulo <= len(angulos_lista):
                                angulo_escolhido_pelo_usuario = angulos_lista[escolha_num_angulo-1]
                                print(f"Você escolheu o ângulo: '{angulo_escolhido_pelo_usuario}'")
                                prosseguir_para_proxima_etapa = True # Sinaliza para ir para o Agente 3
                                break # Sai do loop de escolha de ângulo
                            else:
                                print(f"Escolha inválida. Digite um número entre 1 e {len(angulos_lista)}.")
                        except ValueError:
                            print("Entrada inválida. Por favor, digite um número.")
                        except Exception as e_input_ang:
                            print(f"Erro inesperado durante o input do ângulo: {e_input_ang}")
                            prosseguir_para_proxima_etapa = False
                            break
                else:
                    print("\n⚠️ Não consegui extrair uma lista clara de ângulos para escolha do Agente 2.")
                    prosseguir_para_proxima_etapa = False
            except Exception as e_proc_ang:
                print(f"\n⚠️ Ocorreu um erro ao tentar processar os ângulos: {e_proc_ang}")
                prosseguir_para_proxima_etapa = False
        else:
            print(f"⚠️ Não foi possível gerar ângulos com o Agente 2.")
            prosseguir_para_proxima_etapa = False


    # --- ETAPA 3: Agente Roteirista Expresso ---
    if prosseguir_para_proxima_etapa and angulo_escolhido_pelo_usuario:
        prosseguir_para_proxima_etapa = False # Reseta para a próxima etapa (se houver)
        print("\n--- Chamando Agente 3 (Roteirista Expresso)... ---")
        roteiro_bruto_do_agente3 = agente_roteirista_expresso(tema_escolhido_pelo_usuario, angulo_escolhido_pelo_usuario)

        if roteiro_bruto_do_agente3:
            print("\n--- ✍️ Resultado do Agente 3 (Roteirista Expresso) ---")
            display(to_markdown(roteiro_bruto_do_agente3))
            prosseguir_para_proxima_etapa = True # Sinaliza para ir para o Agente 4
        else:
            print(f"⚠️ Não foi possível gerar um roteiro bruto com o Agente 3.")
            prosseguir_para_proxima_etapa = False


    # --- ETAPA 4: Agente de Refinamento e Finalização ---
    if prosseguir_para_proxima_etapa and roteiro_bruto_do_agente3: # Verifica se o roteiro bruto existe
        print("\n--- Chamando Agente 4 (Refinamento)... ---")
        roteiro_final_do_agente4 = agente_refinamento_final(tema_escolhido_pelo_usuario, angulo_escolhido_pelo_usuario, roteiro_bruto_do_agente3)
        if roteiro_final_do_agente4:
            print("\n--- ✨ Resultado do Agente 4 (Refinamento e Finalização) ---")
            display(to_markdown(roteiro_final_do_agente4))
            print("\n🎉 Processo concluído! Seu roteiro está pronto! 🎉")
        else:
            print("⚠️ Não foi possível refinar o roteiro com o Agente 4.")
    elif tema_escolhido_pelo_usuario and angulo_escolhido_pelo_usuario and not roteiro_bruto_do_agente3:
        # Se chegamos aqui, significa que Agente 3 falhou, mas as escolhas anteriores foram feitas
        pass # A mensagem de erro do Agente 3 já foi impressa
    elif tema_escolhido_pelo_usuario and not angulo_escolhido_pelo_usuario:
        pass # A mensagem de erro do Agente 2 ou da escolha do ângulo já foi impressa
    elif nicho and not tema_escolhido_pelo_usuario:
        pass # A mensagem de erro do Agente 1 ou da escolha do tema já foi impressa

    print("--------------------------------------------------------------")

🚀 Bem-vindo ao Criador de Roteiros para Vídeos Curtos! 🚀
Este sistema utiliza uma cadeia de 4 agentes para gerar ideias e roteiros.

❓ Por favor, digite o NICHO sobre o qual você quer criar vídeos (ex: culinária vegana, games, reflexologia): futebol

Ótimo! Iniciando o processo para o nicho: 'futebol'

--- Chamando Agente 1 (Pesquisador de Temas)... ---

🔎 Agente Pesquisador de Temas: Buscando temas para o nicho 'futebol'...

--- 💡 Resultado do Agente 1 (Pesquisador de Temas) ---


> Ok, vou usar a ferramenta de busca do Google para encontrar os tópicos mais recentes e relevantes sobre futebol para vídeos curtos (TikTok, Reels) nas últimas 2-4 semanas.
> 
> Com base nas minhas buscas, aqui estão 5 temas em alta no nicho de futebol que parecem promissores para vídeos curtos:
> 
> 1.  **Desafios de Habilidade com a Bola:** Desafios de embaixadinhas, domínio de bola, ou chutes precisos ainda são muito populares. Vídeos curtos mostrando criatividade e habilidade individual têm grande potencial de viralização.
> 2.  **Bastidores e Curiosidades dos Clubes:** Mostrar o dia a dia dos jogadores, treinos, momentos de descontração e curiosidades sobre os times geram grande engajamento. Conteúdo exclusivo e acesso privilegiado são sempre atrativos.
> 3.  **Análise Tática Rápida e Visual:** Explicar jogadas complexas, formações táticas e estratégias de jogo de forma simples e visualmente atraente. Usar gráficos e animações pode facilitar a compreensão e atrair um público mais amplo.
> 4.  **Reações a Gols e Lances Polêmicos:** Gravar reações autênticas a momentos marcantes dos jogos, como golaços, defesas incríveis e lances polêmicos. O elemento surpresa e a emoção genuína podem gerar identificação e compartilhamento.
> 5.  **Conteúdo Humorístico e Paródias:** Criar vídeos engraçados com temas relacionados ao futebol, como imitações de jogadores, situações inusitadas em campo e paródias de músicas e memes populares. O humor é uma ótima forma de atrair atenção e gerar engajamento.


--- Processando temas para escolha... ---

✅ Temas extraídos com sucesso. Por favor, escolha um:
1. Desafios de Habilidade com a Bola
2. Bastidores e Curiosidades dos Clubes
3. Análise Tática Rápida e Visual
4. Reações a Gols e Lances Polêmicos
5. Conteúdo Humorístico e Paródias
Digite o NÚMERO do tema escolhido (1-5): 2
Você escolheu o tema: 'Bastidores e Curiosidades dos Clubes'

--- Chamando Agente 2 (Gerador de Ângulos)... ---

🧠 Agente Gerador de Ângulos (v2): Criando ângulos para o tema 'Bastidores e Curiosidades dos Clubes'...

--- 🧠 Resultado do Agente 2 (Gerador de Ângulos) ---


> **ANGULO X:** O Ritual Secreto Antes do Jogo!
> **DESCRIÇÃO:** Vídeo mostrando o "esquenta" inusitado dos jogadores antes de entrar em campo, com superstições, músicas motivacionais e até dancinhas.
> 
> ---
> **ANGULO X:** Tour Bizarro pelo CT: O Que Ninguém Te Mostra!
> **DESCRIÇÃO:** Revelar os cantos "proibidos" do centro de treinamento, como a sala de jogos ultra competitiva, o mural de fotos constrangedoras e os apelidos internos dos atletas.
> 
> ---
> **ANGULO X:** A Cantina dos Craques: Cardápio Secreto e Pedidos Bizarros!
> **DESCRIÇÃO:** Um dia na cantina do clube, mostrando os pratos preferidos dos jogadores, os pedidos mais estranhos e as receitas secretas que turbinam o desempenho em campo.



--- Processando ângulos para escolha... ---

✅ Ângulos extraídos com sucesso. Por favor, escolha um:
1.  O Ritual Secreto Antes do Jogo!
2.  Tour Bizarro pelo CT: O Que Ninguém Te Mostra!
3.  A Cantina dos Craques: Cardápio Secreto e Pedidos Bizarros!
Digite o NÚMERO do ângulo escolhido (1-3): 3
Você escolheu o ângulo: ' A Cantina dos Craques: Cardápio Secreto e Pedidos Bizarros!'

--- Chamando Agente 3 (Roteirista Expresso)... ---

✍️ Agente Roteirista Expresso (v2): Criando roteiro para o tema 'Bastidores e Curiosidades dos Clubes' com o ângulo ' A Cantina dos Craques: Cardápio Secreto e Pedidos Bizarros!'...

--- ✍️ Resultado do Agente 3 (Roteirista Expresso) ---


> Ok, vamos criar um roteiro viral para um vídeo curto sobre os bastidores da cantina dos clubes de futebol!
> 
> **ROTEIRO DO VÍDEO:**
> 
> **Gancho (3-5s):**
> Já imaginou o que os craques comem pra ter tanta energia? 🤯 A gente invadiu a cantina pra descobrir os segredos!
> 
> **Desenvolvimento (15-25s):**
> Se liga! No cardápio secreto dos clubes, não tem só frango com batata doce, viu? Tem de tudo! Desde pratos personalizados pros atletas que precisam de uma dieta específica, até os famosos "pedidos de última hora" antes do jogo. Adivinha só: rola até umas superstições bizarras! Tem jogador que só come macarrão à bolonhesa, outro que jura que o suco de beterraba dá superpoderes, e aquele que não entra em campo sem o brigadeiro da sorte! E a resenha rola solta! A cantina é palco de muita zoeira, aposta e histórias engraçadas que a gente, infelizmente, não pode contar! 🤫
> 
> **CTA (3-5s):**
> Curtiu saber dos bastidores? Então comenta qual outro segredo de clube você quer que a gente revele! 👇
> 
> ---
> **SUGESTÃO DE LEGENDA (sem hashtags):**
> O que os jogadores comem? 🤫 A gente te conta os segredos da cantina dos clubes! ⚽️
> ---


--- Chamando Agente 4 (Refinamento)... ---

✨ Agente de Refinamento (v2): Polindo o roteiro para 'Bastidores e Curiosidades dos Clubes'...

--- ✨ Resultado do Agente 4 (Refinamento e Finalização) ---


> ---
> **ROTEIRO FINALIZADO E OTIMIZADO:**
> 
> **Gancho (3-5s):**
> E aí, curioso! Já se perguntou o que os craques comem pra voar em campo? 🤯 A gente invadiu a cantina e descobriu cada segredo!
> 
> **Desenvolvimento (15-25s):**
> Esqueça o frango com batata doce! 😜 Nos bastidores, o cardápio é turbinado: tem pratos feitos sob medida para cada atleta, com a nutrição que eles precisam pra dar show. Mas o que ninguém imagina são os "pedidos de última hora"... e as superstições! 😅 Tem jogador que só joga bem se comer macarrão à bolonhesa, outro que toma suco de beterraba achando que vira o Hulk, e não pode faltar o brigadeiro da sorte, né? 🍀 E, claro, a cantina ferve com a resenha, as apostas e as histórias que... bem, o que acontece na cantina, fica na cantina! 🤫
> 
> **CTA (3-5s):**
> Qual outro segredo dos clubes você quer que a gente desvende? Deixa nos comentários! 👇
> 
> ---
> **LEGENDA FINAL E ENGAJADORA:**
> Cardápio secreto dos craques REVELADO! 🤫 Descobrimos o que os jogadores comem pra ter tanta energia em campo! ⚽️ Quer mais bastidores? 👀
> 
> ---
> **HASHTAGS ESTRATÉGICAS:**
> #CantinaDosCraques #BastidoresDoFutebol #CuriosidadesDoFutebol #FutebolBrasileiro #AlimentacaoNoFutebol #JogadoresDeFutebol
> 
> ---
> **SUGESTÕES VISUAIS/SONORAS (BÔNUS):**
> - Visual: Imagens dinâmicas da cantina, pratos sendo preparados, jogadores interagindo (mesmo que ilustrativas/genéricas). Animações de "segredos" sendo revelados.
> - Som: Música de fundo animada e divertida, com efeitos sonoros de "suspense" ao revelar os segredos.
> ---


🎉 Processo concluído! Seu roteiro está pronto! 🎉
--------------------------------------------------------------
